In [1]:
import setup_jwlab
import pandas as pd
import numpy as np

from jwlab.ml_prep import load_ml_df, y_to_binary, load_ml_data
from sklearn.model_selection import cross_val_score

In [2]:
#df = load_ml_df("~/projects/def-jwerker/kjslakov/data/ml_df_sktime.pkl")
df = load_ml_df("/Users/JennMacBook/Desktop/Studies/Animates EEG/7_Data/runningOffline/ml_df_sktime.pkl")

loaded


In [3]:
y = y_to_binary(df.label.values.flatten())
df = df.drop(columns=["label"], axis=1)

In [4]:
np.mean(y==0)

0.5005793742757821

In [5]:
from scipy.stats import entropy, kurtosis, skew
funcs = [np.mean, np.min, np.max, np.var, skew, kurtosis]
dfs_applied = [df.applymap(f) for f in funcs]

In [6]:
from functools import reduce

suffix = 0
def join_dfs(a, b):
    global suffix
    suffix = suffix + 1
    return a.join(b, rsuffix="_%d" % suffix)

df_concat = reduce(join_dfs, dfs_applied)

In [7]:
df_normalized = (df_concat - df_concat.mean()) / df_concat.std()
df_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,50_5,51_5,52_5,53_5,54_5,55_5,56_5,57_5,58_5,59_5
0,-0.272031,1.061153,1.536229,1.610506,-0.100752,1.265364,1.297782,0.029161,2.095260,-1.355529,...,-1.031427,-0.884059,-0.826336,-1.016336,-0.605733,-0.303770,0.750309,-0.789845,0.218234,0.888346
1,0.686243,-0.448588,-1.268795,-0.613641,0.120652,-2.538618,-0.316984,-1.065541,-1.292870,-0.278447,...,-0.114729,0.144532,-0.207249,-0.033365,-0.308429,0.259129,-0.029957,1.387946,0.058398,-0.427152
2,-0.060079,-0.451811,-0.563331,-0.525889,-0.453831,-0.981317,-0.812164,-0.310977,-0.771615,0.033736,...,1.440008,0.668225,0.751713,0.958503,-0.110867,0.573967,0.516879,0.454744,0.100228,0.415634
3,0.403587,0.139478,0.246425,-0.086942,0.536083,1.329740,0.351982,0.487611,0.636028,0.800632,...,-0.890473,0.288876,-0.548790,-0.590093,0.623750,1.007703,-0.100604,2.924539,1.810391,0.016063
4,3.985853,-0.160310,-0.592757,-0.253722,0.040797,-0.005113,-0.066781,-0.603454,-0.785278,-0.298385,...,0.365910,-0.295794,-0.610287,2.072710,-0.125737,-0.742365,-0.610178,1.499587,-0.395984,-0.504842


In [8]:
print(len(y))
print(df_normalized.shape)

1726
(1726, 360)


In [9]:
from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=1000, C=1e-3)

scores = cross_val_score(model, df_normalized, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_normalized, y, test_size=0.2)
model.fit(X_train, y_train)
print("train error: %0.2f" % np.mean(model.predict(X_train) != y_train))
print("test error: %0.2f" % np.mean(model.predict(X_test) != y_test))

[0.50578035 0.47687861 0.46086957 0.43478261 0.49709302]
Accuracy: 0.48 (+/- 0.05)
train error: 0.36
test error: 0.52


In [10]:
np.mean(y_test == 0)

0.49421965317919075

In [9]:
# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV
# parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}]
# svc = SVC()
# model = GridSearchCV(svc, parameters)
# model.fit(df_normalized, y)

# model.cv_results_

In [10]:
# model.best_score_

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(model, df_normalized, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.59555556 0.55555556 0.56666667 0.56888889 0.53571429]
Accuracy: 0.56 (+/- 0.04)


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_normalized, y, test_size=0.2)
model.fit(X_train, y_train)
print("train error: %0.2f" % np.mean(model.predict(X_train) != y_train))
print("test error: %0.2f" % np.mean(model.predict(X_test) != y_test))

train error: 0.00
test error: 0.44


In [16]:
from sklearn.svm import SVC
model = SVC(gamma='scale')

scores = cross_val_score(model, df_normalized, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.58       0.56       0.58       0.58       0.58258929]
Accuracy: 0.58 (+/- 0.02)


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_normalized, y, test_size=0.2)
model.fit(X_train, y_train)
print("train error: %0.2f" % np.mean(model.predict(X_train) != y_train))
print("test error: %0.2f" % np.mean(model.predict(X_test) != y_test))

train error: 0.39
test error: 0.42
